In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "drive/MyDrive/nlp"

Mounted at /content/drive
/content/drive/MyDrive/nlp


In [2]:
!pip install -q -U trl transformers accelerate peft datasets bitsandbytes evaluate git+https://github.com/huggingface/huggingface_hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [92]:
!nvidia-smi # check if GPU is avaiable

Fri May  3 09:12:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   78C    P0              33W /  70W |   5485MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [93]:
import torch
torch.cuda.is_available() # check if GPU is avaiable

True

In [107]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig

model_name = "google-bert/bert-base-uncased"
# model_name = "EMBEDDIA/sloberta"

In [108]:
from datasets import load_dataset

# Task: BoolQ
data_boolq_train = load_dataset("csv", data_files="slovene_superglue/SuperGLUE-HumanT/csv/BoolQ/train.csv")
data_boolq_val = load_dataset("csv", data_files="slovene_superglue/SuperGLUE-HumanT/csv/BoolQ/val.csv")
data_boolq_test = load_dataset("csv", data_files="slovene_superglue/SuperGLUE-HumanT/csv/BoolQ/test.csv")


In [109]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [110]:
def preprocess_data(examples):

  passage_inputs = [f"passage : {x} " for x in examples["passage"]]
  question_inputs = [f"question : {x} " for x in examples["question"]]
  inputs = [passage_input + question_input for passage_input, question_input in zip(passage_inputs, question_inputs)]

  model_inputs = tokenizer(inputs, max_length=400, truncation=True)

  model_inputs["labels"] = examples["label"]
  model_inputs["labels"] = [int(label) for label in model_inputs["labels"]]

  return model_inputs

tokenized_boolq_train = data_boolq_train.map(preprocess_data, remove_columns=['idx', 'passage', 'label', 'question'], batched=True)
tokenized_boolq_val = data_boolq_val.map(preprocess_data,remove_columns=['idx', 'passage', 'label', 'question'], batched=True)

print(tokenized_boolq_val["train"])


#for i in train['input_ids']:
#    print(tokenizer.decode(i))

for i in tokenized_boolq_val["train"]['labels']:
  print(i)


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 18
})
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
0


In [111]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [112]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [113]:
id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

model.config.use_cache = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [114]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

NUM_VIRTUAL_TOKENS = 12

peft_config = PromptTuningConfig(
    peft_type="PROMPT_TUNING",
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,
    num_layers=6,
    token_dim=768,
    num_attention_heads=12,
    tokenizer_name_or_path=model_name #The pre-trained model
)

In [115]:
peft_model = get_peft_model(model, peft_config)
print(peft_model.print_trainable_parameters())

trainable params: 10,754 || all params: 109,494,532 || trainable%: 0.009821495013102572
None


In [116]:
from transformers import TrainingArguments

#change for different models
new_model_name = "/prompt_fine_tuned_boolq_bert"
output_directory = "." + new_model_name

training_args = TrainingArguments(
    output_dir = output_directory,
    learning_rate= 2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    use_cpu=False,
    load_best_model_at_end=True,
    num_train_epochs=6,
)

In [117]:
from transformers import Trainer

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_boolq_train["train"],
    eval_dataset=tokenized_boolq_val["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [118]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.684394,0.500000,0.533333
2,No log,0.670020,0.611111,0.640808
3,No log,0.657153,0.777778,0.777778
4,No log,0.649218,0.833333,0.791398
5,No log,0.645428,0.833333,0.791398
6,No log,0.644361,0.833333,0.791398


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=72, training_loss=0.6935620837741427, metrics={'train_runtime': 34.1211, 'train_samples_per_second': 16.178, 'train_steps_per_second': 2.11, 'total_flos': 101984087483616.0, 'train_loss': 0.6935620837741427, 'epoch': 6.0})

In [119]:
# save model to hub
model_location = "tjasad" + new_model_name
trainer.push_to_hub(model_location)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


events.out.tfevents.1714725909.53ae7b07d069.855.12:   0%|          | 0.00/17.0k [00:00<?, ?B/s]

events.out.tfevents.1714725834.53ae7b07d069.855.10:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

Upload 21 LFS files:   0%|          | 0/21 [00:00<?, ?it/s]

events.out.tfevents.1714725776.53ae7b07d069.855.9:   0%|          | 0.00/450 [00:00<?, ?B/s]

...

events.out.tfevents.1714727599.f1df7c01f60e.2080.11:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

events.out.tfevents.1714727637.f1df7c01f60e.2080.12:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

events.out.tfevents.1714727708.f1df7c01f60e.2080.13:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tjasad/prompt_fine_tuned_boolq_bert/commit/063850a9a863cc184e7a3134f968bd3361a5f4d1', commit_message='tjasad/prompt_fine_tuned_boolq_bert', commit_description='', oid='063850a9a863cc184e7a3134f968bd3361a5f4d1', pr_url=None, pr_revision=None, pr_num=None)

In [120]:
trainer.evaluate()

{'eval_loss': 0.6443609595298767,
 'eval_accuracy': 0.8333333333333334,
 'eval_f1': 0.7913978494623656,
 'eval_runtime': 0.5301,
 'eval_samples_per_second': 33.956,
 'eval_steps_per_second': 5.659,
 'epoch': 6.0}

In [123]:
# Example
text="passage : Bankovec za 20 evrov – Zaenkrat obstaja le ena celotna serija evrskih bankovcev, vendar pa izhaja nova serija, ki bo podobna sedanji. Evropska centralna banka bo pravočasno naznanila, kdaj bodo bankovci iz prve serije izgubili status zakonitega plačilnega sredstva. question : Ali je bankovec za 20 evrov iz prve serije še vedno zakonito plačilno sredstvo?"

classifier = pipeline("sentiment-analysis", model='tjasad/prompt_fine_tuned_boolq')
classifier(text)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.5154208540916443}]